In [1]:
########################################################
# A few resources I used:
# https://developers.google.com/kml/articles/geotagsimple
# * Copyright 2008 Google Inc. All Rights Reserved.
# 
# Reads the EXIF headers from geo-tagged photos and creates a KML file with
# a PhotoOverlay element for each file. Requires the open source EXIF.py file
# downloadable at:
# http://sourceforge.net/projects/exif-py/
#   __author__ = 'mmarks@google.com (Mano Marks)'
#   
########################################################
# Photos-gps-to-csv-1
# Purpose: 
# - Writes .csv or text files with filename and gps from photo. 
# - Built off photos-to-kml-draft2

# ###
# Outline:
# - For each file with .jpg extension,
# -    Read properties in header
# -    WRITE full file path, GPS data IF GPS data present, 
# -      *Not yet implemented: add to a single? kml file
# ###

import exifread as EXIF
import os
import sys
import csv 

In [2]:
def GetJPGFile(file_name, root, file_list):
    """ Chooses only to open .jpg files
    Handles opening the file.
    Args:
    file_name: the name of the file to get
    Returns:
    A file     """

    the_file = None
    extension = os.path.splitext(file_name)[1]
    
    if extension == '.JPG':
        fullfilename = os.path.join(root,file_name)
        file_list.append(fullfilename)
        try:
            the_file = open(fullfilename, 'rb')
        except IOError:
            the_file = None

    return the_file, file_list



def GetHeaders(the_file):
    """Handles getting the EXIF headers and returns them as a dict.
    Args:
    the_file: A file object
    Returns:
    a dict mapping keys corresponding to the EXIF headers of a file.
    """
    data = EXIF.process_file(the_file)  # , 'UNDEF', False, False, False
    return data



def DmsToDecimal(degree_num, degree_den, minute_num, minute_den,
                 second_num, second_den):
    """Converts the Degree/Minute/Second formatted GPS data to decimal degrees.
    Args:
    degree_num: The numerator of the degree object.
    degree_den: The denominator of the degree object.
    minute_num: The numerator of the minute object.
    minute_den: The denominator of the minute object.
    second_num: The numerator of the second object.
    second_den: The denominator of the second object.
    Returns:
    A deciminal degree.
    """
    degree = float(degree_num)/float(degree_den)
    minute = float(minute_num)/float(minute_den)/60
    second = float(second_num)/float(second_den)/3600
    return degree + minute + second

def GetGps(data):
    """Parses out the GPS coordinates from the file.
    Args:
    data: A dict object representing the EXIF headers of the photo.
    Returns:
    A tuple representing the latitude, longitude, and altitude of the photo.
    """
    # Error here handled earlier, in main loop
    lat_dms = data['GPS GPSLatitude'].values
    long_dms = data['GPS GPSLongitude'].values
    latitude = DmsToDecimal(lat_dms[0].num, lat_dms[0].den,
                            lat_dms[1].num, lat_dms[1].den,
                            lat_dms[2].num, lat_dms[2].den)

    longitude = DmsToDecimal(long_dms[0].num, long_dms[0].den,
                            long_dms[1].num, long_dms[1].den,
                            long_dms[2].num, long_dms[2].den)
    
    if data['GPS GPSLatitudeRef'].printable == 'S': latitude *= -1
    if data['GPS GPSLongitudeRef'].printable == 'W': longitude *= -1
    altitude = None
    try:
        alt = data['GPS GPSAltitude'].values[0]
        altitude = alt.num/alt.den
        if data['GPS GPSAltitudeRef'] == 1: altitude *= -1
    except KeyError:
        altitude = 0
    return latitude, longitude, altitude


# In[14]:

def CreateCsvDoc(csv_doc, file_name, the_file, file_iterator):
    ''' Writes Csv file'''
    if os.path.exists(csv_doc):
        append_write = 'a'  #append if it already exists
    else: 
        append_write = 'w'  #make a new file if not.

    data = GetHeaders(the_file)
    latitude, longitude, altitude = GetGps(data)
    fields = [latitude,longitude,altitude,file_name]
    with open(csv_doc, append_write) as csvf:
        writer = csv.writer(csvf)
        writer.writerow(fields)

In [3]:
# Input necessary (usrcode == 1), input hardcoded (2)
usrcode = 1

if usrcode ==1: 
    usr_path = raw_input("Enter directory path:")
    the_file = str(usr_path)
else: 
    usr_path = 'C:\Users\\nvmille1\OneDrive\Research\Landuse'
   

Enter directory path:C:\Users\nvmille1\OneDrive\Pictures\Grand Canyon Photos pt II


In [4]:
#Prepare the csv file
#Make a single .kml file in the User path named by the folder. 
csvname = usr_path.split('\\')[-1] + '-photo-GPS-list'
fullcsvname = '{}\{}.csv'.format(usr_path,csvname)
print 'csv name: {}'.format(csvname)

file_iterator = 0
filenames = []


for root, dirs, files in os.walk(usr_path):
    if file_iterator % 25 == 0:
        print 'file iter: {}'.format(file_iterator)
    for f in files:
        fullfilename = os.path.join(root,f)
        #Only adds jpgs to filenames, only goes through full loop if jpg
        the_file, filenames = GetJPGFile(f, root, filenames)  
        if the_file is None:
            #print 'file is none: file iterator {}'.format(file_iterator)
            continue

        try: 
            #Write csv 
            CreateCsvDoc(fullcsvname, fullfilename, the_file, file_iterator)
        except KeyError:
            filenames.remove(fullfilename)
            the_file.close()
            #print "file: {} \ntdata KeyError 'GPS GPSLatitude': file iterator {}".format(fullfilename,file_iterator)
            continue
            
        file_iterator += 1    
        #Close the_file you've opened
        the_file.close()

print 'Finished writing GPS coordinates from {} photos to {}.'.format(file_iterator, csvname)

csv name: Grand Canyon Photos pt II-photo-GPS-list
file iter: 0
Finished writing GPS coordinates from 0 photos to Grand Canyon Photos pt II-photo-GPS-list.
